# Run Main.py then Run next lines for Bokeh Interactive plot

In [8]:
%run main.py



reading from the following configuration files: 

  ['./config/data/text_files.yaml', './config/data/pdf_files.yaml']




Reading in 2 file(s) from ./config/data/text_files.yaml






Reading in 54 file(s) from ./config/data/pdf_files.yaml


doc_names

 ['adventure', 'pride_and_prejudice', 'Addendum to the report of the Islamic Maghreb', 'Administrative instructions', 'AQ Accounting Ledger', 'Draft of a letter to subordinates', 'Duties of administrators', 'General instructions', 'In regard to the money that is in Sudan', 'Instructions to subordinates', 'Jihad in Pakistan', 'Kind brother', 'Letter about efforts in other regions', 'Letter about logistics', 'Letter about matter of the Islamic Maghreb', 'Letter confirming receipt of money', 'Letter directing allocation of funds', 'Letter from Abdallah Abu Zayd Abd-al-Hamid to Abu Mus ab Abd-al-Wadud', 'Letter from Abu Yahya', 'Letter from Abu-Abdallah al-Halabi', 'Letter from Atiyah to Abu Basir', 'Letter of guidance on leadership', 'Let





Running the following algorithm: 
TFIDF 




algorithm: kmeans

result:

 (5, Counter({0: 23, 2: 15, 1: 12, 3: 3, 4: 3}))



algorithm: bag_of_words

result:

 {'interlop': 1, 'artilleri': 1, 'almond': 1, 'vanquish': 4, 'rge': 1, 'crazi': 1, 'icat': 1, 'cheap': 1, 'land': 53, 'heavi': 1, 'batt': 1, 'alk': 1, 'assess': 6, 'servic': 5, 'ween': 1, 'brick': 1, 'novemb': 2, 'fighter': 5, 'compens': 1, 'pretti': 1, 'safar': 3, 'fallen': 1, 'hoto': 1, 'mouth': 1, 'accep': 2, 'criteria': 2, 'gulf': 21, 'chest': 5, 'astonish': 1, 'bhambher': 2, 'madin': 1, 'nur': 1, 'issuanc': 2, 'repli': 12, 'advers': 2, 'oth': 4, 'hesit': 7, 'chechnya': 1, 'desol': 2, 'ecam': 1, 'kno': 2, 'linkag': 1, 'loos': 2, 'ﬁthi': 1, 'pt': 1, 'nu': 1, 'lign': 1, 'ment': 3, 'repeat': 14, 'muli': 1, 'arget': 1, 'merci': 111, 'henceforth': 1, 'follo': 2, 'hug': 1, 'ssit': 1, 'owledg': 1, 'greet': 18, 'sprudenc': 1, 'research': 21, 'mistreat': 1, 'brandish': 2, 'forgotten': 2, 'rtilleri': 1, 'indirectli': 2, 'shown': 7,

word_cloud_data.xlsx can be found in the scitext-explorer file and is ready to be used in Tableau


# Bokeh Imports

In [14]:
import pandas as pd

from bokeh.io import output_notebook, output_file, show, curdoc
from bokeh.plotting import figure
from bokeh.layouts import column, row, WidgetBox
from bokeh.models.widgets import CheckboxGroup, Slider, RangeSlider, Tabs, TextInput, Panel, RadioButtonGroup
from bokeh.models import CustomJS, ColumnDataSource, HoverTool, BoxSelectTool, CrosshairTool, SaveTool, Div, PanTool

output_notebook()
output_file('Bokeh_Test.html')
#BOKEH_RESOURCES=inline

Loading BokehJS ...

INFO:bokeh.io.state:Session output file 'Bokeh_Test.html' already exists, will be overwritten.


# Load ScatterPlot DataFrame

In [15]:
# filename = 'Bokeh Resume Test.xlsx'
filename = 'Bokeh_Test.xlsx'

df = pd.read_excel(filename)

cluster_sets = [str(i).zfill(2) for i in range(2,len(df.columns) - 1)]

dataset = dict()
for colname in df.columns:
#     print(colname)
    if colname in ['x','y','docname']:
        dataset[colname.replace(' ','')] = df[colname].tolist()
    else:
        dataset[str(colname)] = [[c[1:-2].split(", '")[0].strip(),c[1:-2].split(", '")[1].strip()] for c in df[colname].tolist()]
#         dataset[str(colname)] = [c[1:-2].split(", '")[1].strip() for c in resumes[colname].tolist()]
#         dataset['ClusterId'] = [c[1:-2].split(", '")[0].strip() for c in resumes[colname].tolist()]
        
## Starting Color (Gray)
dataset['color'] = ['lightgray'] * len(df)

## Starting size
starting_size = 15
dataset['size'] = [starting_size] * len(df)

## Starting size
dataset['ClusterId'] = [0] * len(df)

# Plot ScatterPlot on Bokeh

In [16]:
import random

cluster_sets = [str(i).zfill(2) for i in range(2,len(df.columns) - 1)]

source = ColumnDataSource(dataset)

## Configure the HoverTool popup dialog
hover = HoverTool(tooltips=[('Document Name', '@docname'),('Cluster Id','@ClusterId')])    

## Create a Bokeh figure object
p = figure(plot_width=600, plot_height=400, 
           tools = [hover,BoxSelectTool(),CrosshairTool(),SaveTool(),PanTool()], toolbar_location="above",
           title = 'Bokeh')

# add a circle renderer with a size, color, and alpha
p.circle('x', 'y', size = 'size', source = source,
         line_color = 'dimgray', line_width = 0.1,
         fill_color = 'color', fill_alpha = 0.6)

def size_callback(source=source, window=None):
    
    data = source.data
    new_size = cb_obj.value
    
    data['size'] = [new_size] * len(data['size'])
        
    source.change.emit()
        
def color_callback(source=source, window=None):
    
    print('ColorCALLBACK!!!')
    
    global color_dict
    
    data = source.data
    new_color = cb_obj.active
    
    data['color'] = [c[1] for c in data[str(new_color + 2)]]
    data['ClusterId'] = [c[0] for c in data[str(new_color + 2)]]
        
    source.change.emit()
    
## Slider to set size of dots
size_of_dots = Slider(start = 1, end = 100, step = 1, value = starting_size,
                        title = 'Scatter Plot Document Size', callback=CustomJS.from_py_func(size_callback))

## Radio Buttons to set color of dots
clusters = RadioButtonGroup(labels = cluster_sets, name = 'Number of Clusters', #active = 0,
                            callback=CustomJS.from_py_func(color_callback))

layout = column(size_of_dots, clusters, p)

show(layout)

# Load BOW DataFrame

In [17]:
filename = 'BOW.xlsx'

df = pd.read_excel(filename)

dataset = dict()
for colname in df.columns:
#     print(colname)
    if colname in ['Word','Word Count']:
        dataset[colname.replace(' ','')] = df[colname].tolist()
    else:
        dataset[str(colname)] = [[c[1:-2].split(", '")[0].strip(),c[1:-2].split(", '")[1].strip()] for c in df[colname].tolist()]

# Plot BOW BarGraph

In [18]:
from bokeh.io import show, output_file
from bokeh.models import ColumnDataSource
from bokeh.palettes import Spectral10
from bokeh.plotting import figure

output_file("bag_of_word.html")

yheight = max(dataset['WordCount']) + 10

p = figure(x_range=dataset['Word'], y_range= (0, yheight), plot_height=350, title="Top 10 Terms in Documents",
           toolbar_location=None, tools="")

p.vbar(x= dataset['Word'], top= dataset['WordCount'], width = 0.9, color='purple')

#, width=0.9, color='color', legend=dataset['Word'], source=source

p.xgrid.grid_line_color = None
p.legend.orientation = "horizontal"
p.legend.location = "top_center"

show(p)

INFO:bokeh.io.state:Session output file 'bag_of_word.html' already exists, will be overwritten.
